In [1]:
from generator_class_multi_1226 import * 

2024-02-12 18:30:42.645851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
# trying to get list of url list to work 
import matplotlib.pylab as plt
import numpy as np
import zlib
import glob
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers, callbacks, losses
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import keras.backend as K
import os
from generator_class_multi_1226 import DataGenerator_3output_train, DataGenerator_3output_test
from tensorflow.keras.optimizers import SGD
import json
import tqdm
import argparse
import pickle
import pandas as pd
from tensorflow.keras.regularizers import l2
from weighted_scce import * #custom weighted loss function for class imbalance. 

#GPU/CPU Selection
gpu_setting = 'y'
random.seed(42) # reproducibility 
# Set the number of threads
num_threads = 8
tf.config.threading.set_inter_op_parallelism_threads(num_threads)

### CHANGE PATH (or remove it, I can remake the plots as long as you save the model.) 
plt.style.use('/home/sophiaf/mystyle.mplstyle')

2024-02-23 20:07:07.077664: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_path = '/home/sophiaf/Classification-with-ML/neutrino-classification/CNN/model_save/ResNet_20240204/'
loaded_model = tf.keras.models.load_model(model_path, custom_objects={'WeightedSCCE': WeightedSCCE_for_saving})

2024-02-23 20:07:11.673645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-23 20:07:11.705326: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-23 20:07:11.705639: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-23 20:07:11.706477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

In [ ]:
model = tf.keras.models.load_model('/Users/aaronhiguera/HEP/DUNE/DUNE_CNN/ResNet_20240204/', 
                                   custom_objects={'WeightedSCCE':WeightedSCCE(), 
                                                   'output_loss_weights':output_loss_weights, 'output_losses':output_losses} )

In [24]:
df = pd.read_pickle('/home/sophiaf/pixel_maps_val/preprocessed_filelists/secondbatch_df_testset.pkl')

In [27]:
test_gen = DataGenerator_3output_test(df.iloc[:6400], 64, (200,200),3)

In [28]:
loaded_model.evaluate(test_gen, verbose=1)
# note that the loss function will be messed up. but for what we care about, that is ok. 

100/100 [==============================] - 13s 120ms/step - loss: nan - flavour_loss: 0.2487 - protons_loss: 0.6328 - pions_loss: nan - flavour_accuracy: 0.9070 - protons_accuracy: 0.7291 - pions_accuracy: 0.8616


[nan,
 0.2486628144979477,
 0.6328144073486328,
 nan,
 0.907031238079071,
 0.7290624976158142,
 0.8615624904632568]

In [ ]:
tar -cvf cvn_atmnu.tar.gz /home/sophiaf/Classification-with-ML/neutrino-classification/CNN/model_save/ResNet_20240204

In [37]:
loaded_model.get_layer('whatami')

ValueError: No such layer: whatami. Existing layers are: ['input_1', 'conv2d', 'batch_normalization', 're_lu', 'max_pooling2d', 'conv2d_1', 'batch_normalization_1', 're_lu_1', 'conv2d_2', 'batch_normalization_2', 'add', 're_lu_2', 'conv2d_3', 'batch_normalization_3', 're_lu_3', 'conv2d_4', 'batch_normalization_4', 'conv2d_5', 'add_1', 're_lu_4', 'conv2d_6', 'batch_normalization_5', 're_lu_5', 'conv2d_7', 'batch_normalization_6', 'conv2d_8', 'add_2', 're_lu_6', 'conv2d_9', 'batch_normalization_7', 're_lu_7', 'conv2d_10', 'batch_normalization_8', 'conv2d_11', 'add_3', 're_lu_8', 'conv2d_12', 'batch_normalization_9', 're_lu_9', 'conv2d_13', 'batch_normalization_10', 'conv2d_14', 'add_4', 're_lu_10', 'global_average_pooling2d', 'dense', 'dense_1', 'flavour', 'protons', 'pions'].

In [30]:
for key, value in loaded_model.signatures['serving_default'].inputs.items():
    print('Input key:', key, 'Shape:', value.shape, 'DType:', value.dtype)

TypeError: Cannot iterate over a tensor with unknown first dimension.

In [33]:
loaded_model.signatures['serving_default'].outputs

[<tf.Tensor 'Identity:0' shape=(None, 3) dtype=float32>,
 <tf.Tensor 'Identity_1:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'Identity_2:0' shape=(None, 4) dtype=float32>]

In [ ]:
for key, value in model.signatures['serving_default'].outputs.items():
    print('Output key:', key, 'Shape:', value.shape, 'DType:', value.dtype)

In [3]:
def residual_block(x, filters, kernel_size=3, stride=1):
    # Shortcut connection
    shortcut = x

    # First convolution layer
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Second convolution layer
    x = layers.Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    # If the number of filters has changed, apply a 1x1 convolution to the shortcut
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, padding='same', kernel_initializer='he_normal')(shortcut)
    
    # Add the shortcut to the output
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

In [4]:
n_channels = 3
dimensions = (200,200)

In [5]:
input_shape = (dimensions[0], dimensions[1], n_channels)
inputs = layers.Input(shape=input_shape)

# Initial Convolution Layer
x = layers.Conv2D(32, 7, strides=2, padding='same', kernel_initializer='he_normal')(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

# Residual Blocks
num_blocks = 5  # Increase the number of residual blocks
filters = [32, 64, 128, 256, 512]   # Increase the number of filters in each block

 
for i in range(num_blocks):
    x = residual_block(x, filters[i])

# Global Average Pooling Layer
x = layers.GlobalAveragePooling2D()(x)

# Fully connected (Dense) layers
x = layers.Dense(128, activation='relu', kernel_initializer='he_normal')(x)
x = layers.Dense(64, activation='sigmoid', kernel_initializer='he_normal')(x)

# Output layer, 3 classifications (no nu_tau) plus sub-class cases. 
output_names=['flavour','protons','pions']
output_neurons=[3,4,2]

outputs = [None]*len(output_names)
for i in range(len(outputs)):
    activation='sigmoid' if output_neurons[i]==1 else 'softmax'
    weight_decay = 1e-4
    outputs[i] = layers.Dense(output_neurons[i], use_bias=False, kernel_regularizer=l2(weight_decay),
                       activation=activation, name=output_names[i])(x)
model = models.Model(inputs, outputs)

2024-02-12 18:30:47.493359: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-12 18:30:47.530231: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-12 18:30:47.530555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-12 18:30:47.531604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

In [6]:
class_weights = [{0: 1.48/(1.48+1+1.18), 1: 1./(1.48+1+1.18), 2: 1.18/(1.48+1+1.18)},
                 {0: 1.31/(1.31+1+3+5), 1: 1./(1.31+1+3+5), 2: 3./(1.31+1+3+5), 3: 5./(1.31+1+3+5)},
                 {0: 1./(1.+2), 1: 2./(1+2.)},
                 ]

class_weights_tensors = [tf.constant(list(weights.values()), dtype=tf.float32) for weights in class_weights]

output_losses = {
"flavour": WeightedSCCE_for_saving(class_weight=class_weights_tensors[0]), # "sparse_categorical_crossentropy",
"protons":  WeightedSCCE_for_saving(class_weight=class_weights_tensors[1]),
"pions":  WeightedSCCE_for_saving(class_weight=class_weights_tensors[2]),
}
output_loss_weights = {"flavour": 1.0, "protons": 1.0, "pions": 1.0, }



In [7]:
sgd_optimizer = SGD(learning_rate=1e-4, momentum=0.9)
model.compile(optimizer=sgd_optimizer,
                  loss = output_losses,
                  loss_weights = output_loss_weights,
              metrics=['accuracy'])

In [9]:
temp_path_tf = '/home/sophiaf/temp_model_save_tf/'  
temp_path = '/home/sophiaf/temp_model_save/'  

# model.save(temp_path_tf, save_format='tf')
# model.save(temp_path)

In [10]:

loaded_model = tf.keras.models.load_model(temp_path_tf, custom_objects={'WeightedSCCE_for_saving': WeightedSCCE_for_saving})